# Acquire and Summarize: 
### 1. Acquire data from the cloud database.

You will want to end with a single dataframe. Include the logerror field and all other fields related to the properties that are available. You will end up using all the tables in the database.

Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid. - Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction. (Hint: read the docs for the .duplicated method) - Only include properties that have a latitude and longitude value.

In [3]:
#imports:
import pandas as pd
import numpy as np
import os
import env

In [4]:
# get connection url:
def get_db_url(db, user= env.user, host=env.host, password=env.password):
    """
    This function will:
    - take credentials from env.py file
    - make a connection to the SQL database with given credentials
    - return url connection
    """
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [5]:
# need a table that specifically: max logerror and max transactiondate

In [6]:
# sql query: 
sql_query_2 = '''
Select *
FROM properties_2017 as pr
	JOIN (Select 
	parcelid,
    max(logerror) as logerrorr, 
    max(transactiondate) as transactiondate
From
	predictions_2017
Group by parcelid) as predictions_2017 
	on predictions_2017.parcelid = pr.parcelid
	Left JOIN airconditioningtype as air 
		on pr.airconditioningtypeid = air.airconditioningtypeid
	Left Join architecturalstyletype as ar
		on  pr.architecturalstyletypeid = ar.architecturalstyletypeid
	Left Join buildingclasstype as bu
		on pr.buildingclasstypeid = bu.buildingclasstypeid
	Left Join heatingorsystemtype as he
		on pr.heatingorsystemtypeid = he.heatingorsystemtypeid
	Left Join propertylandusetype po
		on pr.propertylandusetypeid = po.propertylandusetypeid
	Left Join storytype as st
		on pr.storytypeid = st.storytypeid
	Left Join typeconstructiontype as ty
		on pr.typeconstructiontypeid = ty.typeconstructiontypeid
    Left Join unique_properties as up
		on pr.parcelid = up.parcelid
	Where 
		YEAR(predictions_2017.transactiondate) = 2017
        AND (pr.latitude IS NOT NULL AND pr.longitude IS NOT NULL);'''

In [7]:
sql_query = """
SELECT prop.*, 
       pred.logerror, 
       pred.transactiondate, 
       air.airconditioningdesc, 
       arch.architecturalstyledesc, 
       build.buildingclassdesc, 
       heat.heatingorsystemdesc, 
       landuse.propertylandusedesc, 
       story.storydesc, 
       construct.typeconstructiondesc 

FROM   properties_2017 prop  
       INNER JOIN (SELECT parcelid,
       					  logerror,
                          Max(transactiondate) transactiondate 
                   FROM   predictions_2017 
                   GROUP  BY parcelid, logerror) pred
               USING (parcelid) 
       LEFT JOIN airconditioningtype air USING (airconditioningtypeid) 
       LEFT JOIN architecturalstyletype arch USING (architecturalstyletypeid) 
       LEFT JOIN buildingclasstype build USING (buildingclasstypeid) 
       LEFT JOIN heatingorsystemtype heat USING (heatingorsystemtypeid) 
       LEFT JOIN propertylandusetype landuse USING (propertylandusetypeid) 
       LEFT JOIN storytype story USING (storytypeid) 
       LEFT JOIN typeconstructiontype construct USING (typeconstructiontypeid) 
WHERE  prop.latitude IS NOT NULL 
       AND prop.longitude IS NOT NULL AND transactiondate <= '2017-12-31' 
"""

In [8]:
# make sure the sql query works:
df = pd.read_sql(sql_query, get_db_url('zillow'))
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1727539,14297519,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,6.059063e+13,0.025595,2017-01-01,None,None,None,None,Single Family Residential,None,None
1,1387261,17052889,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,6.111001e+13,0.055619,2017-01-01,None,None,None,None,Single Family Residential,None,None
2,11677,14186244,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,6.059022e+13,0.005383,2017-01-01,None,None,None,None,Single Family Residential,None,None
3,2288172,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,6.037300e+13,-0.103410,2017-01-01,None,None,None,Central,Single Family Residential,None,None
4,1970746,10887214,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,6.037124e+13,0.006940,2017-01-01,Central,None,None,Central,Condominium,None,None


In [9]:
# are there anu duplicates?
duplicate_rows = df[df.duplicated(keep='first')]
duplicate_rows

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc


In [13]:
def read_zillow():
    '''
    This function return a zillow based on the following requirements:
    - Only unclude propeties with a transaction in 2017
    - Include only the last transaction for each property
    - Include only the last zestimate error and date of transaction
    - Only include properties that have a latitude and longitude value.
    '''
    sql_query: sql_query = '''
    SELECT prop.*, 
       pred.logerror, 
       pred.transactiondate, 
       air.airconditioningdesc, 
       arch.architecturalstyledesc, 
       build.buildingclassdesc, 
       heat.heatingorsystemdesc, 
       landuse.propertylandusedesc, 
       story.storydesc, 
       construct.typeconstructiondesc 

FROM   properties_2017 prop  
       INNER JOIN (SELECT parcelid,
       					  logerror,
                          Max(transactiondate) transactiondate 
                   FROM   predictions_2017 
                   GROUP  BY parcelid, logerror) pred
               USING (parcelid) 
       LEFT JOIN airconditioningtype air USING (airconditioningtypeid) 
       LEFT JOIN architecturalstyletype arch USING (architecturalstyletypeid) 
       LEFT JOIN buildingclasstype build USING (buildingclasstypeid) 
       LEFT JOIN heatingorsystemtype heat USING (heatingorsystemtypeid) 
       LEFT JOIN propertylandusetype landuse USING (propertylandusetypeid) 
       LEFT JOIN storytype story USING (storytypeid) 
       LEFT JOIN typeconstructiontype construct USING (typeconstructiontypeid) 
WHERE  prop.latitude IS NOT NULL 
       AND prop.longitude IS NOT NULL AND transactiondate <= '2017-12-31' '''
    
    # return the data frame
    return pd.read_sql(sql_query, get_db_url('zillow'))

In [14]:
def get_zillow_data():
    '''
    This function reads in data from a codeup database, writes the data to a csv file if a 
    local file does not exist, and returns a df
    '''
    
    if os.path.isfile('zillow.csv'):
        
        #if csv file exists, read in data from csv file 
        df = pd.read_csv('zillow.csv', index_col = 0)
        
    else:
        
        # read fresh data from db into a dataframe
        df = read_zillow()
        
        #write dataframe to a csv file
        df.to_csv('zillow.csv')
    
    return df

In [16]:
get_zillow_data()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1727539,14297519,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,6.059063e+13,0.025595,2017-01-01,None,None,None,None,Single Family Residential,None,None
1,1387261,17052889,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,6.111001e+13,0.055619,2017-01-01,None,None,None,None,Single Family Residential,None,None
2,11677,14186244,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,6.059022e+13,0.005383,2017-01-01,None,None,None,None,Single Family Residential,None,None
3,2288172,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,6.037300e+13,-0.103410,2017-01-01,None,None,None,Central,Single Family Residential,None,None
4,1970746,10887214,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,6.037124e+13,0.006940,2017-01-01,Central,None,None,Central,Condominium,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77569,2864704,10833991,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,6.037132e+13,-0.002245,2017-09-20,Central,None,None,Central,Condominium,None,None
77570,673515,11000655,NaN,NaN,NaN,2.0,2.0,NaN,6.0,2.0,...,6.037101e+13,0.020615,2017-09-20,None,None,None,Central,Single Family Residential,None,None
77571,2968375,17239384,NaN,NaN,NaN,2.0,4.0,NaN,NaN,2.0,...,6.111008e+13,0.013209,2017-09-21,None,None,None,None,Single Family Residential,None,None
77572,1843709,12773139,1.0,NaN,NaN,1.0,3.0,NaN,4.0,1.0,...,6.037434e+13,0.037129,2017-09-21,Central,None,None,Central,Single Family Residential,None,None


### 2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [42]:
df.describe()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,parcelid,logerrorr,airconditioningtypeid,architecturalstyletypeid,buildingclasstypeid,heatingorsystemtypeid,propertylandusetypeid,storytypeid,typeconstructiontypeid,parcelid
count,7.738000e+04,7.738000e+04,24953.000000,206.000000,50.000000,77380.000000,77380.000000,15.000000,49671.000000,76771.000000,...,7.738000e+04,77380.000000,24953.000000,206.000000,15.000000,49439.000000,77380.000000,50.0,222.000000,7.738000e+04
mean,1.495126e+06,1.300715e+07,1.813289,7.388350,679.720000,2.299134,3.053489,3.933333,6.534638,2.316871,...,1.300715e+07,0.017046,1.813289,7.388350,3.933333,3.920447,261.826350,7.0,6.040541,1.300715e+07
std,8.609057e+05,3.481368e+06,2.967894,2.734542,689.703546,0.996657,1.139103,0.258199,1.721933,0.979761,...,3.481368e+06,0.169728,2.967894,2.734542,0.258199,3.592789,5.141231,0.0,0.557285,3.481368e+06
min,3.490000e+02,1.071186e+07,1.000000,2.000000,38.000000,0.000000,0.000000,3.000000,1.000000,1.000000,...,1.071186e+07,-4.655420,1.000000,2.000000,3.000000,1.000000,31.000000,7.0,4.000000,1.071186e+07
25%,7.520500e+05,1.153830e+07,1.000000,7.000000,273.000000,2.000000,2.000000,4.000000,6.000000,2.000000,...,1.153830e+07,-0.024122,1.000000,7.000000,4.000000,2.000000,261.000000,7.0,6.000000,1.153830e+07
50%,1.497870e+06,1.253155e+07,1.000000,7.000000,515.000000,2.000000,3.000000,4.000000,6.000000,2.000000,...,1.253155e+07,0.006760,1.000000,7.000000,4.000000,2.000000,261.000000,7.0,6.000000,1.253155e+07
75%,2.240480e+06,1.421184e+07,1.000000,7.000000,796.500000,3.000000,4.000000,4.000000,8.000000,3.000000,...,1.421184e+07,0.039363,1.000000,7.000000,4.000000,7.000000,266.000000,7.0,6.000000,1.421184e+07
max,2.982274e+06,1.676893e+08,13.000000,21.000000,3560.000000,18.000000,16.000000,4.000000,12.000000,18.000000,...,1.676893e+08,5.262999,13.000000,21.000000,4.000000,24.000000,275.000000,7.0,13.000000,1.676893e+08


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77380 entries, 0 to 77379
Data columns (total 77 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            77380 non-null  int64  
 1   parcelid                      77380 non-null  int64  
 2   airconditioningtypeid         24953 non-null  float64
 3   architecturalstyletypeid      206 non-null    float64
 4   basementsqft                  50 non-null     float64
 5   bathroomcnt                   77380 non-null  float64
 6   bedroomcnt                    77380 non-null  float64
 7   buildingclasstypeid           15 non-null     float64
 8   buildingqualitytypeid         49671 non-null  float64
 9   calculatedbathnbr             76771 non-null  float64
 10  decktypeid                    614 non-null    float64
 11  finishedfloor1squarefeet      6023 non-null   float64
 12  calculatedfinishedsquarefeet  77184 non-null  float64
 13  f

In [44]:
df.shape

(77380, 77)

### 3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [45]:
# missing values in columns: 
df.isna().sum()

id                              0
parcelid                        0
airconditioningtypeid       52427
architecturalstyletypeid    77174
basementsqft                77330
                            ...  
storytypeid                 77330
storydesc                   77330
typeconstructiontypeid      77158
typeconstructiondesc        77158
parcelid                        0
Length: 77, dtype: int64

In [46]:
# missing values in rows: 
df.isna().sum(axis=1)

0        42
1        39
2        40
3        37
4        33
         ..
77375    33
77376    38
77377    38
77378    36
77379    39
Length: 77380, dtype: int64

In [47]:
df_nulls = df.copy().T
df_nulls

,0,1,2,3,4,5,6,7,8,9,...,77370,77371,77372,77373,77374,77375,77376,77377,77378,77379
id,1727539,1387261,11677,2288172,1970746,1447245,781532,870991,1246926,1585097,...,1635173,1684937,1642435,1373391,2274245,2864704,673515,2968375,1843709,1187175
parcelid,14297519,17052889,14186244,12177905,10887214,17143294,12095076,12069064,12790562,11542646,...,12892446,12666457,10858613,10722691,12412492,10833991,11000655,17239384,12773139,12826780
airconditioningtypeid,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,1.0,NaN
architecturalstyletypeid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
basementsqft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
storytypeid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
storydesc,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
typeconstructiontypeid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
typeconstructiondesc,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [48]:
df_nulls['num_rows_missing'] = df.isna().sum(axis=0)
df_nulls

,0,1,2,3,4,5,6,7,8,9,...,77371,77372,77373,77374,77375,77376,77377,77378,77379,num_rows_missing
id,1727539,1387261,11677,2288172,1970746,1447245,781532,870991,1246926,1585097,...,1684937,1642435,1373391,2274245,2864704,673515,2968375,1843709,1187175,0
parcelid,14297519,17052889,14186244,12177905,10887214,17143294,12095076,12069064,12790562,11542646,...,12666457,10858613,10722691,12412492,10833991,11000655,17239384,12773139,12826780,0
airconditioningtypeid,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,...,NaN,1.0,1.0,NaN,1.0,NaN,NaN,1.0,NaN,52427
architecturalstyletypeid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77174
basementsqft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
storytypeid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77330
storydesc,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,77330
typeconstructiontypeid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77158
typeconstructiondesc,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,77158


In [49]:
df_nulls['pct_rows_missing'] = (df.isna().sum(axis=0)/df.shape[0])

In [50]:
df_null = df_nulls[['num_rows_missing','pct_rows_missing']]
df_null

,num_rows_missing,pct_rows_missing
id,0,0.000000
parcelid,0,0.000000
airconditioningtypeid,52427,0.677526
architecturalstyletypeid,77174,0.997338
basementsqft,77330,0.999354
...,...,...
storytypeid,77330,0.999354
storydesc,77330,0.999354
typeconstructiontypeid,77158,0.997131
typeconstructiondesc,77158,0.997131


In [23]:
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    pct_missing = num_missing / rows
    cols_missing = pd.DataFrame({'number_missing_rows': num_missing, 'percent_rows_missing': pct_missing})
    return cols_missing

In [24]:
nulls_by_col(df)

,number_missing_rows,percent_rows_missing
id,0,0.000000
parcelid,0,0.000000
airconditioningtypeid,52568,0.677650
architecturalstyletypeid,77368,0.997344
basementsqft,77524,0.999355
...,...,...
buildingclassdesc,77559,0.999807
heatingorsystemdesc,28005,0.361010
propertylandusedesc,0,0.000000
storydesc,77524,0.999355


# Prepare: 

### 1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer.

In [26]:
# Restrict df to only properties that meet single unit criteria

single_use = [261, 262, 263, 264, 266, 268, 273, 276, 279]
df = df[df.propertylandusetypeid.isin(single_use)]
    
    
# Restrict df to only those properties with at least 1 bath & bed and >350 sqft area
df = df[(df.bedroomcnt > 0) & (df.bathroomcnt > 0) & ((df.unitcnt<=1)|df.unitcnt.isnull()) & (df.calculatedfinishedsquarefeet>350)]


In [27]:
def handle_missing_values(df, prop_required_column = .5, prop_required_row = .70):
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

In [28]:
df = handle_missing_values(df)